In [ ]:
import matplotlib.pyplot as plt  # type: ignore
from dotenv import load_dotenv
import sqlalchemy as sq
import seaborn as sns  # type: ignore
import pandas as pd  # type: ignore
import numpy as np
import os, sys

sys.path.append("../")
from Shared.DataService import DataService

In [ ]:
load_dotenv()
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")

In [ ]:
if (
    PG_DB is None
    or PG_ADDR is None
    or PG_PORT is None
    or PG_USER is None
    or PG_PW is None
):
    raise ValueError("Environment variables not set")

db = DataService(PG_DB, PG_ADDR, int(PG_PORT), PG_USER, PG_PW)
conn = db.connect()

In [ ]:
ergotQuery = sq.text("SELECT * FROM agg_ergot_samples;")

ergotDF = pd.read_sql_query(ergotQuery, conn)

In [ ]:
moistureQuery = sq.text(
    "SELECT soil_moisture_min, soil_moisture_max, soil_moisture_mean, year, district FROM agg_soil_moisture;"
)

moistureDF = pd.read_sql_query(moistureQuery, conn)

In [ ]:
ab_df = moistureDF.loc[moistureDF["district"] >= 4800]
sk_df = moistureDF.loc[
    (moistureDF["district"] >= 4700) | (moistureDF["district"] < 4800)
]
mb_df = moistureDF.loc[moistureDF["district"] < 4700]

In [ ]:
db.cleanup()

In [ ]:
final_df = moistureDF.merge(ergotDF, on=["year", "district"])

In [ ]:
final_df.drop(columns=["district", "year"], inplace=True)

In [ ]:
# Compute the correlation matrix
corr = final_df.corr()

In [ ]:
corr

In [ ]:
sns.set_theme(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=0.5,
    vmin=-0.5,
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
)

In [ ]:
sns.pairplot(final_df)
plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
fig.suptitle("Soil Moisture (m^3/m^3)")


ax1.hist(sk_df["soil_moisture_min"], alpha=0.5, label="sk")
ax1.hist(ab_df["soil_moisture_min"], alpha=0.5, label="ab")
ax1.hist(mb_df["soil_moisture_min"], alpha=0.5, label="mb")
ax1.title.set_text("min")
ax1.legend(loc="upper left")

ax2.hist(sk_df["soil_moisture_mean"], alpha=0.5, label="sk")
ax2.hist(ab_df["soil_moisture_mean"], alpha=0.5, label="ab")
ax2.hist(mb_df["soil_moisture_mean"], alpha=0.5, label="mb")
ax2.title.set_text("mean")
ax2.legend(loc="upper left")

ax3.hist(sk_df["soil_moisture_max"], alpha=0.5, label="sk")
ax3.hist(ab_df["soil_moisture_max"], alpha=0.5, label="ab")
ax3.hist(mb_df["soil_moisture_max"], alpha=0.5, label="mb")
ax3.title.set_text("max")
ax3.legend(loc="upper left")